In [10]:
import yfinance as yf
import pandas as pd

# define the variable "btc" as the yf ticker function, and the argument as the BTC ticker.
btc = yf.Ticker("BTC-USD") 

# define the variable "btc_data" as the yf historical price retrieval function, with the start, end date and interval as arguments.
btc_data = btc.history(start="2020-01-01", end="2025-04-20", interval="1d")

# create the dataframe, using pandas function .DataFrame taking the variable "btc_data" as argument. This fills the dataframe with historical bitcoin prices as per defined in previous line. 
btc_df = pd.DataFrame(btc_data)

# clean up the dataset a little bit, removing volume, dividend and stock splits, as these are irrelevant.
btc_df = btc_df.drop(columns=["Volume", "Dividends", "Stock Splits"])

# print the head, or the start, of the dataframe.
print(btc_df.head())

                                  Open         High          Low        Close
Date                                                                         
2020-01-01 00:00:00+00:00  7194.892090  7254.330566  7174.944336  7200.174316
2020-01-02 00:00:00+00:00  7202.551270  7212.155273  6935.270020  6985.470215
2020-01-03 00:00:00+00:00  6984.428711  7413.715332  6914.996094  7344.884277
2020-01-04 00:00:00+00:00  7345.375488  7427.385742  7309.514160  7410.656738
2020-01-05 00:00:00+00:00  7410.451660  7544.497070  7400.535645  7411.317383
